## TODO LIST

- Get mbp fbp geo coordinates
- Make sure 1910, 1920 have the same number of dummy columns (dropping sdxn for now because of this)

In [1]:
import pandas as pd
import numpy as np
from metric_learn import MMC, ITML

In [ ]:
#Randomly generate false matches
def create_false(year: int, rand: int) -> pd.core.frame.DataFrame:
    return ((pd.read_csv(f"R:/JoePriceResearch/record_linking/data/census_compact/{year}/ind_ark{year}.csv"))
             .sample(n=1000000, random_state=rand).reset_index(drop=True))
years = [1910, 1920]
false_matches = pd.concat([create_false(years[0], 94), create_false(years[1], 42)], axis=1)

In [ ]:
false_matches = false_matches.drop(["index1910", "index1920"], axis=1)

In [ ]:
false_matches.shape

In [ ]:
#Create match sample
false_matches["y"] = 0
true_matches = pd.read_stata("R:/JoePriceResearch/record_linking/data/crosswalks/arkhints1910_1920_small.dta")
is_false = false_matches.merge(true_matches, how="left", on=["ark1910", "ark1920"], indicator=True)["_merge"] == "left_only"
in_arkhints1 = false_matches.merge(true_matches, how="left", on=["ark1910"], indicator=True)["_merge"] == "both"
in_arkhints2 = false_matches.merge(true_matches, how="left", on=["ark1920"], indicator=True)["_merge"] == "both"
false_matches = false_matches.loc[is_false & (in_arkhints1 | in_arkhints2), :]
true_matches["y"] = 1


In [ ]:
false_matches.shape

In [ ]:
true_matches = true_matches.sample(n=500000).reset_index(drop=True)

In [ ]:
matches = pd.concat([false_matches, true_matches], axis=0).reset_index(drop=True)
data = [matches[[f"ark{years[0]}", "y"]], matches[[f"ark{years[1]}", "y"]]]

In [ ]:
print(matches.head())
print(matches.tail())

In [ ]:
data[0].to_csv("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/weight_training_arks_1910.csv", index=None)
data[1].to_csv("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/weight_training_arks_1920.csv", index=None)

In [ ]:
del data

Run stata code here

In [23]:
cols_to_drop = ["household1910", "midinit", "cohort1", "cohort2", "bp", "mbp", "fbp", "county_int", "state_int", "last_sdxn", "first_sdxn"]
cols_to_rename = {"ark1910" : "ark", "index1910": "index"}
data1910 = pd.read_stata("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/training_weights_1910.dta", index_col="num")
data1910 = (data1910.assign(immigrant=(~data1910.immigration.isnull()).astype(int))
                    .assign(immigration=data1910.immigration.mask(data1910.immigration.isnull(), 0))
                    .drop(cols_to_drop, axis=1)
                    .rename(cols_to_rename, axis=1))
data1910.columns = [f"{i}_1910" for i in data1910.columns]

In [24]:
cols_to_drop = ["household1920", "midinit", "cohort1", "cohort2", "bp", "mbp", "fbp", "county_int", "state_int", "last_sdxn", "first_sdxn"]
cols_to_rename = {"ark1920" : "ark", "index1920": "index"}
data1920 = pd.read_stata("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/training_weights_1920.dta", index_col="num")
data1920 = (data1920.assign(immigrant=(~data1920.immigration.isnull()).astype(int))
                    .assign(immigration=data1920.immigration.mask(data1920.immigration.isnull(), 0))
                    .drop(cols_to_drop, axis=1)
                    .rename(cols_to_rename, axis=1))
data1920.columns = [f"{i}_1920" for i in data1920.columns]

In [25]:
data = data1910.join(data1920, how="inner").dropna()

In [26]:
print(data.shape)
print(data.loc[data.y_1910 == data.y_1920, "y_1910"].shape)

(852540, 840)
(852540,)


In [27]:
y = data.y_1910.rename("y")

In [28]:
indices = data[["ark_1910", "ark_1920", "index_1910", "index_1920"]]
data = data.drop(["ark_1910", "ark_1920", "index_1910", "index_1920", "y_1910", "y_1920"], axis=1)

In [29]:
#Save data
pd.concat([data, y, indices], axis=1).to_csv("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/metric_learning_data.csv", index=None)

In [2]:
data = pd.read_csv("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/metric_learning_data.csv", nrows=10000).drop(["ark_1910", "ark_1920", "index_1910", "index_1920"], axis=1)

In [30]:
data = data.loc[(data.female_1910 != -1) & (data.race_1910 != -1) & (data.rel_1910 != -1) & (data.marstat_1910 != -1) & (data.first_init_1910 != -1) & (data.last_init_1910 != -1) & (data.female_1920 != -1) & (data.race_1920 != -1) & (data.rel_1920 != -1) & (data.marstat_1920 != -1) & (data.first_init_1920 != -1) & (data.last_init_1920 != -1), :]

In [44]:
data = data.sample(n=20000)

In [45]:
#Separate similar and dissimilar pairs
#data = pd.concat([data, y], axis=1)
sim_pairs_1910 = data.loc[data.y == 1, :"immigrant_1910"].reset_index(drop=True)
sim_pairs_1920 = data.loc[data.y == 1, "marstat_1920":"immigrant_1920"].reset_index(drop=True)
dissim_pairs_1910 = data.loc[data.y == 0, :"immigrant_1910"].reset_index(drop=True)
dissim_pairs_1920 = data.loc[data.y == 0, "marstat_1920":"immigrant_1920"].reset_index(drop=True)

In [46]:
#Rename columns
sim_pairs_1910.columns = [i.replace("_1910", "") for i in sim_pairs_1910.columns]
sim_pairs_1920.columns = [i.replace("_1920", "") for i in sim_pairs_1920.columns]

dissim_pairs_1910.columns = [i.replace("_1910", "") for i in dissim_pairs_1910.columns]
dissim_pairs_1920.columns = [i.replace("_1920", "") for i in dissim_pairs_1920.columns]

In [47]:
X = pd.concat([sim_pairs_1910, sim_pairs_1920, dissim_pairs_1910, dissim_pairs_1920], ignore_index=True, sort=False)

In [48]:
#Create dummies
dummies_list = ["marstat", "race", "rel", "first_init", "last_init"]
X[dummies_list] = X[dummies_list].astype("category")
X = pd.concat([X.drop(dummies_list, axis=1), 
                  pd.get_dummies(X[dummies_list], prefix=dummies_list)], axis=1)

In [49]:
#Create pair indices
import numpy as np
a = np.array(sim_pairs_1910.index)
b = np.array(sim_pairs_1920.index    + sim_pairs_1910.shape[0])
c = np.array(dissim_pairs_1910.index + sim_pairs_1910.shape[0] + sim_pairs_1920.shape[0])
d = np.array(dissim_pairs_1920.index + sim_pairs_1910.shape[0] + sim_pairs_1920.shape[0] + dissim_pairs_1910.shape[0])
constraints = (a, b, c, d)

In [10]:
del data, sim_pairs_1910, sim_pairs_1920, dissim_pairs_1910, dissim_pairs_1920, a, b, c, d
import gc; gc.collect()

56

## ITML

In [50]:
from metric_learn import ITML
itml = ITML()
itml = itml.fit(X.values, constraints)

In [51]:
import pickle as pkl
with open("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/metric_transformer2.itml", "wb") as file:
    pkl.dump(itml, file)

In [54]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.concat([pd.DataFrame(X.columns), pd.DataFrame(np.diagonal(itml.transformer()))], axis=1))

                   0          0
0         birth_year   0.997732
1        immigration   0.000712
2             female  23.119884
3         first_vec2   0.661860
4         first_vec3   0.698280
5         first_vec4   0.715732
6         first_vec5   0.644950
7         first_vec6   0.654289
8         first_vec7   0.666933
9         first_vec8   0.685699
10        first_vec9   0.722271
11       first_vec10   0.649856
12       first_vec11   0.727848
13       first_vec12   0.627441
14       first_vec13   0.804641
15       first_vec14   0.710718
16       first_vec15   0.672062
17       first_vec16   0.672818
18       first_vec17   0.644906
19       first_vec18   0.623184
20       first_vec19   0.705530
21       first_vec20   0.650774
22       first_vec21   0.673409
23       first_vec22   0.688978
24       first_vec23   0.649854
25       first_vec24   0.706764
26       first_vec25   0.668495
27       first_vec26   0.656656
28       first_vec27   0.688597
29       first_vec28   0.698683
30      

In [55]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.concat([pd.DataFrame(X.columns), pd.DataFrame(itml.transformer())], axis=1))

                 0         0         1          2         3         4    \
0         birth_year  0.997732  0.000171   0.340997  0.254440  0.067788   
1        immigration  0.000000  0.000712   4.253913  0.045109 -0.005116   
2             female  0.000000  0.000000  23.119884  0.052964  0.490846   
3         first_vec2  0.000000  0.000000   0.000000  0.661860 -0.036577   
4         first_vec3  0.000000  0.000000   0.000000  0.000000  0.698280   
5         first_vec4  0.000000  0.000000   0.000000  0.000000  0.000000   
6         first_vec5  0.000000  0.000000   0.000000  0.000000  0.000000   
7         first_vec6  0.000000  0.000000   0.000000  0.000000  0.000000   
8         first_vec7  0.000000  0.000000   0.000000  0.000000  0.000000   
9         first_vec8  0.000000  0.000000   0.000000  0.000000  0.000000   
10        first_vec9  0.000000  0.000000   0.000000  0.000000  0.000000   
11       first_vec10  0.000000  0.000000   0.000000  0.000000  0.000000   
12       first_vec11  0.0

## MMC

In [ ]:
mmc = MMC(max_iter=1000, diagonal=True, verbose=True)
mmc = mmc.fit(X.values, constraints)

In [ ]:
mmc.transformer()

In [ ]:
weights = pd.concat([pd.DataFrame(X.columns), pd.DataFrame(np.diagonal(mmc.transformer()))], axis=1)

In [ ]:
weights.columns = ["feature", "weight"]

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(weights)

In [ ]:
import pickle as pkl
with open("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/approx_nearest_neighbor/weights.array", "wb") as file:
    pkl.dump(mmc.transformer(), file)

In [ ]:
"""
import sys
sys.path.append("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/Splycer")
from preprocessing import EuclideanDistance, BooleanMatch

params = {"variables": [["birth_year"], ["event_lat", "event_lon"], ["bplace_lat", "bplace_lon"], [f"first_vec{i}" for i in range(2, 202)], [f"last_vec{i}" for i in range(2,202)]], "new_cols": ["birth_year_dist", "geodist", "bp_geodist", "first_vec_dist", "last_vec_dist"], "years": ["1910", "1920"]}
euc = EuclideanDistance(**params)
data = euc.fit_transform(data)

params = {"vars_to_match": ["marstat", "race", "rel", "first_init", "last_init", "immigrant"]}
match = BooleanMatch(**params)
data = match.fit_transform(data)

for year in ["1910", "1920"]:
    cols_to_drop = ["marstat", "birth_year", "race", "rel", "female", "first_init", "last_init", "event_lat", "event_lon", "bplace_lat", "bplace_lon", "immigrant"]
    vec_list = [f"first_vec{i}" for i in range(2,202)]
    cols_to_drop.extend(vec_list)
    cols_to_drop = [f"{i}_{year}" for i in cols_to_drop]
    data = data.drop(cols_to_drop, axis=1)
    
from sklearn.preprocessing import normalize

features = normalize(data)

from sklearn.linear_model import Lasso, LinearRegression, LogisticRegression

reg = LinearRegression()
lass = Lasso()
reg = reg.fit(features, y)
reg.coef_

lass = lass.fit(features, y)
lass.coef_

logit = LogisticRegression()
logit = logit.fit(features, y)
logit.coef_
"""